In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Parsing

In [ ]:
import os
import aiohttp
import asyncio
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Настройки
BASE_URL = "https://docs.moodle.org/500/en/Main_page"
DOMAIN_PREFIX = "https://docs.moodle.org"
OUTPUT_DIR = "/content/drive/MyDrive/moodle_docs9"
CONCURRENCY = 10
MAX_PAGES = 20000

# Создаём директории, если не существуют
os.makedirs(OUTPUT_DIR, exist_ok=True)

visited_urls = set()
semaphore = asyncio.Semaphore(CONCURRENCY)

# Фразы для фильтрации нежелательных страниц
irrelevant_phrases = [
     "Talk:",
    "Category:",
    "Special:",
    "Template:",

]

def is_valid_link(href):
    # Фильтруем по ссылкам, исключаем медиафайлы и неинформативные ссылки
    excluded_exts = [".png", ".jpg", ".jpeg", ".gif", ".svg", ".ico", ".css", ".js"]
    if any(href.lower().endswith(ext) for ext in excluded_exts):
        return False
    if not href.startswith("/500/en/"):
        return False
    if any(x in href for x in ["Special:", "Category:", "Talk:", "Template:", "action=edit", "oldid=", "index.php?"]):
        return False
    return True

async def fetch(session, url):
    try:
        async with session.get(url) as resp:
            if resp.status == 200:
                return await resp.text()
            print(f"⚠️ HTTP {resp.status} для {url}")
            return None
    except Exception as e:
        print(f"❌ Ошибка запроса {url}: {e}")
        return None

async def fetch_and_parse(session, url, to_visit):
    async with semaphore:
        if url in visited_urls or len(visited_urls) >= MAX_PAGES:
            return
        visited_urls.add(url)

        html = await fetch(session, url)
        if not html:
            return

        soup = BeautifulSoup(html, "html.parser")

        title = soup.find("h1")
        title_text = title.get_text(strip=True) if title else "Untitled"
        filename = url.split("/")[-1] or "index"
        filename = filename.replace("?", "_") + ".md"

        content_div = soup.find("div", {"id": "content"})
        if content_div:
            text = content_div.get_text(separator="\n", strip=True)

            # Фильтрация только для НЕ главной страницы
            if url != BASE_URL:
                matched_phrase = None
                for phrase in irrelevant_phrases:
                    if phrase in text:
                        matched_phrase = phrase
                        break

                if matched_phrase is not None:
                    print(f"⛔ Пропущено из-за '{matched_phrase}': {url}")
                    return

                if len(text) < 300:
                    print(f"⛔ Пропущено из-за короткого текста (<300 символов): {url}")
                    return

            filepath = os.path.join(OUTPUT_DIR, filename)
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(f"# {title_text}\n\n{text}")
            print(f"✅ Сохранено: {filename}")

        # Собираем новые ссылки для обхода
        for a in soup.select("a[href]"):
            href = a['href']
            if is_valid_link(href):
                full_url = urljoin(DOMAIN_PREFIX, href)
                if full_url not in visited_urls:
                    to_visit.add(full_url)

async def crawl_all():
    async with aiohttp.ClientSession() as session:
        to_visit = {BASE_URL}
        while to_visit and len(visited_urls) < MAX_PAGES:
            current_batch = list(to_visit)
            to_visit.clear()
            tasks = [fetch_and_parse(session, url, to_visit) for url in current_batch]
            await asyncio.gather(*tasks)

# Для запуска в Jupyter / Colab (если нужно)
import nest_asyncio
nest_asyncio.apply()

# Запуск основного процесса
await crawl_all()


✅ Сохранено: Main_page.md
✅ Сохранено: Activities.md
✅ Сохранено: Add_students.md
✅ Сохранено: Admin_quick_guide.md
✅ Сохранено: Managing_a_Moodle_course.md
✅ Сохранено: Add_users.md
✅ Сохранено: Moodle_app_notifications.md
✅ Сохранено: MoodleDocs:About.md
✅ Сохранено: New_for_mobile.md
✅ Сохранено: Course_homepage.md
✅ Сохранено: New_for_students.md
✅ Сохранено: Moodle_app_features.md
✅ Сохранено: Quiz_activity.md
✅ Сохранено: Installing_plugins.md
✅ Сохранено: Teacher_quick_guide.md
✅ Сохранено: Adding_a_new_course.md
✅ Сохранено: Table_of_Contents.md
✅ Сохранено: Tracking_progress.md
✅ Сохранено: Assignment_activity.md
✅ Сохранено: Site_appearance.md
✅ Сохранено: Moodle_app_additional_features.md
✅ Сохранено: Forum_activity.md
✅ Сохранено: Workshop_activity.md
✅ Сохранено: New_for_teachers.md
✅ Сохранено: Moodle_app.md
✅ Сохранено: Creating_Moodle-app-friendly_courses.md
✅ Сохранено: MoodleDocs:General_disclaimer.md
✅ Сохранено: Managing_a_Moodle_site.md
✅ Сохранено: Working_with_fi

In [ ]:
import os

folder_path = '/content/drive/MyDrive/moodle_docs9'  # укажи свой путь

file_count = sum(len(files) for _, _, files in os.walk(folder_path))
print(f"Всего файлов: {file_count}")

Всего файлов: 2312
